<a href="https://colab.research.google.com/github/marreapato/Deep_Learning_Course/blob/main/mllib_pyspartfimllib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/mlcrs/

!ls

/content/gdrive/MyDrive/mlcrs
access_log.txt			      KFoldCrossValidation.ipynb     Percentiles.ipynb
breakfast.jpg			      KMeans.ipynb		     PoliticsExercise.ipynb
bridge.jpg			      LinearRegression.ipynb	     PolynomialRegression.ipynb
bunny.jpg			      mammographic_masses.data.txt   Python101.ipynb
castle.jpg			      mammographic_masses.names.txt  regression.txt
ConditionalProbabilityExercise.ipynb  mammo_masses_project.ipynb     Seaborn.ipynb
ConditionalProbabilitySolution.ipynb  MatPlotLib.ipynb		     SparkDecisionTree.py
CovarianceCorrelation.ipynb	      MeanMedianExercise.ipynb	     SparkKMeans.py
DecisionTree.ipynb		      MeanMedianMode.ipynb	     SparkLinearRegression.py
DeepLearningProject.ipynb	      ml-100k			     SparkPCA.py
DeepLearningProject-Solution.ipynb    MLCourse.zip		     StdDevVariance.ipynb
distancerecomendation-KNN.ipynb       mlcrsItemBasedCF.ipynb	     subset-small.tsv
Distributions.ipynb		      mlcrsQ-Learning.ipynb	     SVC.ipynb
emails				      mlcrs_Sim

In [5]:
data_dir = "/content/gdrive/MyDrive/mlcrs/regression.txt"

In [4]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [662 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,600 kB]
Get:13 http://archive.u

In [9]:
from __future__ import print_function

from pyspark.ml.regression import LinearRegression

from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors

if __name__ == "__main__":

    # Create a SparkSession (Note, the config section is only for Windows!)
    spark = SparkSession.builder.config("spark.sql.warehouse.dir", "file:///C:/temp").appName("LinearRegression").getOrCreate()

    # Load up our data and convert it to the format MLLib expects.
    inputLines = spark.sparkContext.textFile(data_dir)
    data = inputLines.map(lambda x: x.split(",")).map(lambda x: (float(x[0]), Vectors.dense(float(x[1]))))

    # Convert this RDD to a DataFrame
    colNames = ["label", "features"]
    df = data.toDF(colNames)

    # Note, there are lots of cases where you can avoid going from an RDD to a DataFrame.
    # Perhaps you're importing data from a real database. Or you are using structured streaming
    # to get your data.

    # Let's split our data into training data and testing data
    trainTest = df.randomSplit([0.5, 0.5])
    trainingDF = trainTest[0]
    testDF = trainTest[1]

    # Now create our linear regression model
    lir = LinearRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

    # Train the model using our training data
    model = lir.fit(trainingDF)

    # Now see if we can predict values in our test data.
    # Generate predictions using our linear regression model for all features in our
    # test dataframe:
    fullPredictions = model.transform(testDF).cache()

    # Extract the predictions and the "known" correct labels.
    predictions = fullPredictions.select("prediction").rdd.map(lambda x: x[0])
    labels = fullPredictions.select("label").rdd.map(lambda x: x[0])

    # Zip them together
    predictionAndLabel = predictions.zip(labels).collect()

    # Print out the predicted and actual values for each point
    for prediction in predictionAndLabel:
      print(prediction)


    # Stop the session
    spark.stop()

(-1.7864102683320184, -2.58)
(-1.8282388178966213, -2.36)
(-1.5214961210895341, -2.27)
(-1.563324670654137, -2.26)
(-1.5214961210895341, -2.17)
(-1.319324798193954, -2.12)
(-1.3681247726859906, -2.09)
(-1.4169247471780273, -2.07)
(-1.3750961976134244, -1.94)
(-1.291439098484219, -1.91)
(-1.3123533732665202, -1.88)
(-1.2774962486293513, -1.8)
(-1.1729248747178442, -1.75)
(-1.152010599935543, -1.74)
(-1.012582101386867, -1.67)
(-1.1380677500806753, -1.66)
(-1.1310963251532415, -1.65)
(-1.2774962486293513, -1.64)
(-1.1938391495001457, -1.61)
(-1.1241249002258078, -1.59)
(-1.0056106764594333, -1.58)
(-1.145039175008109, -1.58)
(-1.082296350661205, -1.57)
(-0.9568107019673966, -1.48)
(-1.012582101386867, -1.47)
(-1.1032106254435063, -1.42)
(-0.8661821779107572, -1.37)
(-0.7894965037089854, -1.3)
(-1.0195535263143007, -1.29)
(-0.8104107784912868, -1.26)
(-0.838296478201022, -1.26)
(-0.7546393790718166, -1.24)
(-0.8313250532735882, -1.22)
(-0.8313250532735882, -1.2)
(-0.8592107529833234, -1.1

In [7]:
df

DataFrame[label: double, features: vector]

IndentationError: unexpected indent (<ipython-input-8-53e4c9b4fd5b>, line 2)